# Laboratorium 2


### Konfiguracja

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes

    def set_axis(self, ax):
        self.ax = ax
        
    def next(self, event):
        self.i = (self.i + 1) % len(scenes)
        self.draw()

    def prev(self, event):
        self.i = (self.i - 1) % len(scenes)
        self.draw()
        
    def draw(self):
        self.ax.clear()
        for collection in scenes[self.i].points:
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), c=collection.color, marker=collection.marker)
        for collection in scenes[self.i].lines:
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale()
        plt.draw()

### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [93]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points = [], color = None, marker = None):
        self.points = np.array(points)
        self.color = color
        self.marker = marker

class LinesCollection:
    def __init__(self, lines = [], color = None):
        self.color = color
        self.lines = lines
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        if self.color:
            return mcoll.LineCollection(self.lines, colors=mcolors.to_rgba(self.color))
        else:
            return mcoll.LineCollection(self.lines)
            


class Plot:
    def __init__(self, scenes = [], json = None):
        if json is None:
            self.scenes = scenes
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
        
    def __configure_buttons(self, callback):
        plt.subplots_adjust(bottom=0.2)
        axprev = plt.axes([0.6, 0.05, 0.15, 0.075])
        axnext = plt.axes([0.76, 0.05, 0.15, 0.075])
        bnext = Button(axnext, 'Następny')
        bnext.on_clicked(callback.next)
        bprev = Button(axprev, 'Poprzedni')
        bprev.on_clicked(callback.prev)
        return [bprev, bnext]

    def draw(self):
        plt.close()
        callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons(callback)
        callback.set_axis(plt.axes())
        plt.show()
        callback.draw()
        
    def toJSON(self):
        return js.dumps([{"points": [pointCol.points.tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])
    

### Przykład użycia

In [94]:
%matplotlib notebook

scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], 'green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]], 'orange')]), 
        Scene([PointsCollection([(1, 2), (-15, 1.5), (2, -1)], 'red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], 'black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])])]

plot = Plot(scenes)
plot.draw() 


<IPython.core.display.Javascript object>

# Rozwiązanie

## zad 1) 

In [95]:
%matplotlib notebook
import math
def point_on_cycle(R, t):
    return [R * math.cos((math.pi / 2) * t), R * math.sin((math.pi / 2) * t)]

def point_on_side(side):
    switcher = {
        0: [np.random.uniform(-10, 10), 10],
        1: [np.random.uniform(-10, 10), -10],
        2: [-10, np.random.uniform(-10, 10),],
        3: [10,np.random.uniform(-10, 10)],
    }
    return switcher.get(side)

def rand_linear(n, low, high, pa, pb):
    result = []
    for i in range(n):
        x = np.random.uniform(low, high)
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        result.append([x, y])
    return result

s_a = [0, 0]
s_b = [10, 0]
s_c = [10, 10]
s_d = [0, 10]
diagonal_1 = rand_linear(20, 0, 10, s_a, s_c)
diagonal_2 = rand_linear(20, 0, 10, s_b, s_d)
axis_y = [[0, np.random.uniform(0, 10)] for _ in range(25)]
axis_x = [[np.random.uniform(0, 10), 0] for _ in range(25)]

a = np.random.uniform(-100, 100, (100, 2))
b = np.random.uniform(0, 4, 100)
b = list(map(lambda t: point_on_cycle(10, t), b))
c = list(map(lambda side: point_on_side(side), np.random.randint(0, 4, 100)))
d = [s_a, s_b, s_c, s_d] + diagonal_1 + diagonal_2 + axis_x + axis_y

## zad 2) 

In [96]:
%matplotlib notebook
scenes=[Scene([PointsCollection(a, 'red')]), 
       Scene([PointsCollection(b, 'green')]),
        Scene([PointsCollection(c, 'blue')]),
        Scene([PointsCollection(d, 'purple')]),]

plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

## zad 3) 

In [97]:
%matplotlib notebook

def generate_cloud(points_number, low, high):
    cloud = np.random.uniform(low, high, (points_number, 2))
    return cloud


def generate_cloud_collection(points_number, low, high, color):
    cloud = generate_cloud(points_number, low, high)
    return PointsCollection(cloud, color)


def generate_circle(points_number, middle, radius):
    def point_on_cycle(R, middle, t):
        return [R * math.cos((math.pi / 2) * t) + middle[0], R * math.sin((math.pi / 2) * t) + middle[1]]

    parameters_set = np.random.uniform(0, 4, points_number)
    circle = list(map(lambda t: point_on_cycle(radius, middle, t), parameters_set))
    return circle


def generate_circle_collection(points_number, middle, radius, color):
    circle = generate_circle(points_number, middle, radius)
    return PointsCollection(circle, color)


def generate_rectangle(points_number, vertices_list):
    def rand_linear_point(pa, pb):
        if pa[0] == pb[0]:
            return [pa[0], np.random.uniform(pa[1], pb[1])]
        x = np.random.uniform(pa[0], pb[0])
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        return [x, y]
        
    
    def point_on_side(side, vertices_list):
        (p_a, p_b, p_c, p_d) = vertices_list
        switcher = {
            0: rand_linear_point(p_a, p_b),
            1: rand_linear_point(p_b, p_c),
            2: rand_linear_point(p_c, p_d),
            3: rand_linear_point(p_d, p_a),
        }
        return switcher.get(side)
        
    rectangle = list(map(lambda side: point_on_side(side, vertices_list), np.random.randint(0, 4, points_number)))
    return rectangle

def generate_rectangle_collection(points_number, vertices_list, color):
    rectangle = generate_rectangle(points_number, vertices_list)
    return PointsCollection(rectangle, color)


def generate_square(vertices_list, axis_number, diagonal_number):
    (p_a, p_b, p_c, p_d) = vertices_list
    def rand_linear_point(pa, pb):
        if pa[0] == pb[0]:
            return [pa[0], np.random.uniform(pa[1], pb[1])]
        x = np.random.uniform(pa[0], pb[0])
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        return [x, y]
    
    diagonal_1 = [ rand_linear_point(p_a, p_c) for _ in range(diagonal_number)]
    diagonal_2 = [ rand_linear_point(p_b, p_d) for _ in range(diagonal_number)]
    axis_1 = [ rand_linear_point(p_a, p_b) for _ in range(axis_number)]
    axis_2 = [ rand_linear_point(p_a, p_d) for _ in range(axis_number)]
    
    square = diagonal_1 + diagonal_2 + axis_1 + axis_2 + [p_a, p_b, p_c, p_d]
    return square

def generate_square_collection(vertices_list, axis_number, diagonal_number, color):
    square = generate_square(vertices_list, axis_number, diagonal_number)
    return PointsCollection(square, color)

### test if generators work

In [98]:
%matplotlib notebook


scenes=[Scene([generate_cloud_collection(100, -100, 100, 'red')]), 
       Scene([generate_circle_collection(100, [0, 0], 10, 'green')]),
        Scene([generate_rectangle_collection(100, [[-10, -10], [10, -10], [10, 10], [-10, 10]], 'blue')]),
        Scene([generate_square_collection([[0, 0], [10, 0], [10, 10], [0, 10]], 25, 20, 'purple')]),]
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

### Generate modified sets

In [99]:
%matplotlib notebook

a_prim = generate_cloud(1000, -250, 100)
b_prim = generate_circle(1000, [0, 5], 13)
c_prim = generate_rectangle(1000, [[-13, -7], [12, -5], [14, 11], [-7, 9]])
d_prim = generate_square([[0, 0], [15, 0], [15, 15], [0, 15]], 20, 90)

scenes=[Scene([PointsCollection(a_prim, 'red')]), 
       Scene([PointsCollection(b_prim, 'green')]), 
        Scene([PointsCollection(c_prim, 'blue')]), 
        Scene([PointsCollection(d_prim, 'purple')]),] 
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

## zad 4)

In [ ]:
%matplotlib notebook

def graham(points):
    sorted_points = list.sort(points)